In [3]:
import os
import json
import time
import datetime as dt
import math
import import_ipynb
from data_processing import DataLoader
from build_model import Model
import pandas as pd
import numpy as np
import keras
import tensorflow
# from plotly.offline import iplot
import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)

In [ ]:
def plot_results(predicted_data, true_data,pre_time):
    pre_time = pd.to_datetime(pre_time)

    trace1 = go.Scatter(x=pre_time,
                        y=true_data,
                        mode='lines',
                        name='True',
                        hoverinfo='name',
                        line=dict(
                                    shape='spline'
                                 )
                        )
    
    trace2 = go.Scatter(x=pre_time,
                        y=predicted_data,
                        mode='lines',
                        name='Prediction',
                        hoverinfo='name',
                        line=dict(
                                    shape='spline'
                                 )
                        )
    data = [trace1, trace2]
    layout = go.Layout(title = 'Prediction & True',
#                    yaxis = dict(title = '%'),
                   xaxis = dict(title = 'timestamp')
                  )
    fig = go.Figure(data=data, layout=layout)
    py.offline.plot(fig)
    

In [ ]:
configs = json.load(open('config.json', 'r'))
if not os.path.exists(configs['model']['save_dir']): os.makedirs(configs['model']['save_dir'])
print('import dataset:', configs['data']['filename'])
data = DataLoader(
    filename=os.path.join('data', configs['data']['filename']),
    split1=configs['data']['train_test_split1'],
    split2=configs['data']['train_test_split2'],
    cols=configs['data']['columns'],
    pre_len=24,
    input_timesteps=configs['model']['layers'][0]['input_timesteps'],
    seq_len = configs['data']['sequence_length']
)
configs['model']['layers'][0]['input_timesteps']
train_x, train_y = data.get_train_data()
train_x.shape
val_x,val_y = data.get_val_data()
# train_x, train_y,val_x,val_y = data.get_train_val_data()
# train_x.shape

In [ ]:
x_test, y_test = data.get_test_data()
pre_time = data.get_pre_time()

steps_per_epoch = math.ceil((len(train_x) - configs['data']['sequence_length']) / 
                            configs['training']['batch_size'])
save_dir = configs['model']['save_dir']
# pre_time
y_test.shape

In [ ]:
score = {}
models_dir = []
for i in range(1):
    print('Starting training %s Model'%(i+1))
    model = Model()
    model.build_model(configs)
    
    time_now = dt.datetime.now().strftime('%Y%m%d-%H%M%S')
    save_dir = configs['model']['save_dir']
    save_dir = os.path.join(save_dir, '%s-e%s'%(time_now,str(i)))
    models_dir.append(save_dir)
    os.makedirs(save_dir)
    
    save_fname = os.path.join(save_dir, 'e%s.h5'%(str(i)))
    log_fname = save_dir
    
    model.model_to_json(save_dir)
    
    fname = os.path.join(save_dir, 'model_information.json')
    
    model.train_generator(
            data_gen=data.training_batch_generator(
            batch_size=configs['training']['batch_size']
        ),
            val_gen=data.val_batch_generator(
            batch_size=configs['training']['batch_size']
        ),
        epochs=configs['training']['epochs'],
        batch_size=configs['training']['batch_size'],
        steps_per_epoch=steps_per_epoch,
        log_fname=log_fname,
        save_fname=save_fname
    )
    
    score_ = model.evaluate(val_x, val_y)
    print("loss:", score_)
    score[save_fname] = score_
    
    
    with open(fname, "w") as to:
        with open("./config.json",'r') as original:
            m = json.loads(original.read())
            m['loss']=score_
            json_str=json.dumps(m)
            to.write(json_str)
            
    print('[Model] Store model_information at %s' % fname)
    
filename_best = min(score,key=score.get)
print(filename_best)
model.load_model(filename_best)
predictions = model.predict_point_by_point(x_test)

In [ ]:
plot_results(predicted_data=predictions, 
             true_data=np.reshape(y_test, (y_test.size,)),
             pre_time=np.reshape(pre_time,(pre_time.size,)))